## **Importation des librairies**

In [40]:
import os
import cv2
import random
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

In [58]:
seed_constant = 25
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)

## **Importation des données**

Nous allons utiliser le jeu de données suivant  [UCF50 - Action Recognition Dataset](https://www.crcv.ucf.edu/data/UCF50.php).  
Ce dataset contient des vidéos réalistes extraites de Youtube.

In [42]:
!wget --no-check-certificate https://www.crcv.ucf.edu/data/UCF50.rar

--2022-06-04 20:19:06--  https://www.crcv.ucf.edu/data/UCF50.rar
Resolving www.crcv.ucf.edu (www.crcv.ucf.edu)... 132.170.214.127
Connecting to www.crcv.ucf.edu (www.crcv.ucf.edu)|132.170.214.127|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 3233554570 (3.0G) [application/rar]
Saving to: ‘UCF50.rar’

UCF50.rar           100%[===================>]   3.01G  13.2MB/s    in 3m 56s  

2022-06-04 20:23:04 (13.1 MB/s) - ‘UCF50.rar’ saved [3233554570/3233554570]



In [45]:
!unrar x UCF50.rar data/

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Extracting  data/UCF50/HorseRace/v_HorseRace_g16_c03.avi                  23%  OK 
Extracting  data/UCF50/HorseRace/v_HorseRace_g16_c04.avi                  23%  OK 
Extracting  data/UCF50/HorseRace/v_HorseRace_g16_c05.avi                  23%  OK 
Extracting  data/UCF50/HorseRace/v_HorseRace_g17_c01.avi                  23%  OK 
Extracting  data/UCF50/HorseRace/v_HorseRace_g17_c02.avi                  23%  OK 
Extracting  data/UCF50/HorseRace/v_HorseRace_g17_c03.avi                  23%  OK 
Extracting  data/UCF50/HorseRace/v_HorseRace_g17_c04.avi                  23%  OK 
Extracting  data/UCF50/HorseRace/v_HorseRace_g17_c05.avi                  23%  OK 
Extracting  data/UCF50/HorseRace/v_HorseRace_g18_c01.avi                  23%  OK 
Extracting  data/UCF50/HorseRace/v_HorseRace_g18_c02.avi                  23%  OK 
Ex

## **Prétraitement des données**

### Inititialisation des variables

In [46]:
# Définition de la hauteur et de la largeur auquelles chaque video frame du dataset sera redimensionné
IMAGE_HEIGHT , IMAGE_WIDTH = 64, 64

# Définition du nombre de frames qui seront transmises au modèle en une seule séquence pour chaque vidéo
SEQUENCE_LENGTH = 20

# Répertoire contenant le jeu de données
DATASET_DIR = "/content/data/UCF50"

# Liste des noms des classes utilisés lors de l'entrainement
CLASSES_LIST = ["Punch", "YoYo", "Swing", "HorseRace"]

### Création d'une fonction pour extraire, redimensionner et normaliser les frames

Nous allons créer une fonction qui permet d'extraire les frames d'une vidéo, après les avoir redimensionné et normalisé.  

Elle prend en argument le chemin d'accès à la vidéo de laquelle nous souhaitons extraire les frames et retourne une liste contenant les frames redimensionnés et normalisés.

In [47]:
def frames_extraction(video_path):

    # Liste pour stocker les video frames
    frames_list = []
    
    # Lecture du fichier vidéo en utilisant l'objet VideoCapture
    video_reader = cv2.VideoCapture(video_path)

    # Nombre total de frames dans la vidéo
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calcul de l'interval après lequel chaque frame sera ajouté à la liste
    skip_frames_window = max(int(video_frames_count/SEQUENCE_LENGTH), 1)

    # Parcours des video frames
    for frame_counter in range(SEQUENCE_LENGTH):

        # Définition de la position actuelle du frame de la vidéo
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)

        # Lecture du frame à partir de la vidéo 
        success, frame = video_reader.read() 

        if not success:
            break

        # Redimensionnement du frame
        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
        # Normalisation du frame redimensionné en le divisant par 255 pour que la valeur de chaque pixel soit comprise entre 0 et 1
        normalized_frame = resized_frame / 255
        
        # Ajout du frame normalisé à la liste des frames
        frames_list.append(normalized_frame)
    
    video_reader.release()

    return frames_list

Ensuite, nous allons créer une fonction qui parcourra toutes les classes spécifiées dans la constante CLASSES_LIST et appellera la fonction frame_extraction() sur chaque fichier vidéo des classes sélectionnées et renverra les images (features), l'index de classe (labels) , et le chemin du fichier vidéo (video_files_paths).

In [48]:
def create_dataset():

    features = []
    labels = []
    video_files_paths = []
    
    for class_index, class_name in enumerate(CLASSES_LIST):   
        print(f'Extraction des données de la Classe: {class_name}')
        
        # liste des fichiers vidéos présents dans le répertoire du class name
        files_list = os.listdir(os.path.join(DATASET_DIR, class_name))
        for file_name in files_list:
            
            video_file_path = os.path.join(DATASET_DIR, class_name, file_name)
            # Extraction des frames du fichier vidéo
            frames = frames_extraction(video_file_path)
            # Prendre en compte uniquement les frames de longueur égale à SEQUENCE_LENGTH
            if len(frames) == SEQUENCE_LENGTH:
                features.append(frames)
                labels.append(class_index)
                video_files_paths.append(video_file_path)
    #Conversion des listes en numpy arrays
    features = np.asarray(features)
    labels = np.array(labels)  
    
    return features, labels, video_files_paths

In [49]:
# Création du dataset
features, labels, video_files_paths = create_dataset()

Extraction des données de la Classe: Punch
Extraction des données de la Classe: YoYo
Extraction des données de la Classe: Swing
Extraction des données de la Classe: HorseRace


In [50]:
# Conversion des labels en one-hot-encoded vectors
one_hot_encoded_labels = to_categorical(labels)

## **Partitionnement des données en ensembles d'entrainement et de test**



In [51]:
features_train, features_test, labels_train, labels_test = train_test_split(features, one_hot_encoded_labels,
                                                                            test_size = 0.25, shuffle = True,
                                                                            random_state = 23)

## **Implémentation du modèle**

### Construction du modèle

In [61]:
def create_model():
   
    model = Sequential()

    model.add(ConvLSTM2D(filters = 4, kernel_size = (3, 3), activation = 'tanh',
                         recurrent_dropout=0.2, return_sequences=True, input_shape = (SEQUENCE_LENGTH, IMAGE_HEIGHT, IMAGE_WIDTH, 3)))
    
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same'))
    model.add(TimeDistributed(Dropout(0.2)))
    
    model.add(ConvLSTM2D(filters = 8, kernel_size = (3, 3), activation = 'tanh', recurrent_dropout=0.2, return_sequences=True))
    
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same'))
    model.add(TimeDistributed(Dropout(0.2)))
    
    model.add(ConvLSTM2D(filters = 14, kernel_size = (3, 3), activation = 'tanh', recurrent_dropout=0.2, return_sequences=True))
    
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same'))
    model.add(TimeDistributed(Dropout(0.2)))
    
    model.add(ConvLSTM2D(filters = 16, kernel_size = (3, 3), activation = 'tanh',recurrent_dropout=0.2, return_sequences=True))
    
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same'))
    
    model.add(Flatten()) 
    
    model.add(Dense(len(CLASSES_LIST), activation = "softmax"))
    
    
    
    model.summary()
    
    return model

In [62]:
conv_model = create_model()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d_17 (ConvLSTM2D)  (None, 20, 62, 62, 4)    1024      
                                                                 
 max_pooling3d_17 (MaxPoolin  (None, 20, 31, 31, 4)    0         
 g3D)                                                            
                                                                 
 time_distributed_12 (TimeDi  (None, 20, 31, 31, 4)    0         
 stributed)                                                      
                                                                 
 conv_lstm2d_18 (ConvLSTM2D)  (None, 20, 29, 29, 8)    3488      
                                                                 
 max_pooling3d_18 (MaxPoolin  (None, 20, 15, 15, 8)    0         
 g3D)                                                            
                                                      

### Compilation et entrainement du modèle

In [63]:
# Create an Instance of Early Stopping Callback
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 10, mode = 'min', restore_best_weights = True)

# Compile the model and specify loss function, optimizer and metrics values to the model
conv_model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

# Start training the model.
convl_model_training_history = conv_model.fit(x = features_train, y = labels_train, epochs = 50, batch_size = 4,
                                                     shuffle = True, validation_split = 0.2, 
                                                     callbacks = [early_stopping_callback])

Epoch 1/50
83/83 [==============================] - 54s 545ms/step - loss: 1.1774 - accuracy: 0.4532 - val_loss: 0.8342 - val_accuracy: 0.6988
Epoch 2/50
83/83 [==============================] - 45s 538ms/step - loss: 0.6053 - accuracy: 0.7855 - val_loss: 0.8039 - val_accuracy: 0.6386
Epoch 3/50
83/83 [==============================] - 44s 530ms/step - loss: 0.4835 - accuracy: 0.8157 - val_loss: 0.5584 - val_accuracy: 0.8434
Epoch 4/50
83/83 [==============================] - 45s 541ms/step - loss: 0.3890 - accuracy: 0.8640 - val_loss: 0.5236 - val_accuracy: 0.8193
Epoch 5/50
83/83 [==============================] - 44s 533ms/step - loss: 0.3767 - accuracy: 0.8580 - val_loss: 0.3452 - val_accuracy: 0.8795
Epoch 6/50
83/83 [==============================] - 44s 526ms/step - loss: 0.1893 - accuracy: 0.9275 - val_loss: 0.3710 - val_accuracy: 0.8434
Epoch 7/50
83/83 [==============================] - 44s 536ms/step - loss: 0.2297 - accuracy: 0.9094 - val_loss: 0.3083 - val_accuracy: 0.8675

### Evaluation du modèle

In [64]:
model_evaluation_history = conv_model.evaluate(features_test, labels_test)

5/5 [==============================] - 0s 78ms/step - loss: 0.6018 - accuracy: 0.8551


### Enregistrement du modèle

In [68]:
model_evaluation_loss, model_evaluation_accuracy = model_evaluation_history

model_file_name = f"model_Loss_{model_evaluation_loss}_Accuracy_{model_evaluation_accuracy}.h5"

conv_model.save(model_file_name)